# El proceso de búsqueda de elementos similares

Repasemos todo el proceso de búsqueda de elementos similares, con el objetivo de buscar, de un conjunto de 10 textos de poesías de Pablo Neruda, cuales son los dos más similares. 

Destacamos las siguientes etapas.
1. Shingling. Esto es, transformar el texto a conjuntos. 
2. Minhashing. Para poder calcular la similitud, computaremos una firma de Minhash para cada uno de los conjuntos, en este caso, cada uno de los shingles de los textos. 
3. (opcional) Ahora podemos estimar la similitud de jaccard como la proporcion de filas en la firma de minhash que es igual.  
4. Para evitar comparar demasiados textos, usamos Locally-Sensitiver Hashing: dividimos la firma de Minhash en b bandas con r filas cada una. 
5. Solo comparamos aquellos pares en las que las r filas de alguna de las bandas sean iguales. Para esto, ¡volvemos a usar una funcion de hash! 

## Shingling

In [23]:
### Para almacenar los distintos textos creamos un diccionario. 
odas = {}

### Vamos a subir cada texto indexado por un keyword distinto en este diccionario. 
### Para hacerlos más legibles, reemplazamos los fin de línea por un espacio. 

with open("oda_alegria.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['alegria']=text

with open("oda_caldillo.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['caldillo']=text

with open("oda_feliz.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['feliz']=text

with open("oda_libro.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['libro']=text

with open("oda_mar.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['mar']=text

with open("oda_poetas.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['poetas']=text

with open("oda_tiempo.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['tiempo']=text

with open("oda_tristeza.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['tristeza']=text


with open("oda_valparaiso.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['valparaiso']=text


with open("oda_vino.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['vino']=text



In [24]:
odas_shingles = {}
k = 3

### iteramos sobre todas las odas
for (name,text) in odas.items(): 
  
    odas_shingles[name] = set()
    
    for i in range(len(text) - k-1):
        shingle = text[i:i+k]
        odas_shingles[name].add(shingle)
        

## Jaccard Similarity


In [5]:
### Lo primero es una funcion que calcula la similitud de Jaccard entre dos conjuntos. 
### Recordemos que se define como la proporcion entre el tamaño de la intersección de los conjuntos, 
### dividido por el tamaño de su unión. 

def jaccard_similarity(set1, set2):
    # Computa la similitud de Jaccard entre dos sets
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

## Hashing

In [6]:
### Para que esta funcion de hash tenga buenas propiedades aleatorias, 
### - p debe ser un primo mayor que n, 
### - a y b deben estar entre 1 y p-1 (e idealmente ser escogidas al azar).

def crear_hash(a, b, p, n):
    def f(x):
        return ((a * x + b) % p) % n
    return f

## Minhashing

Para calcular las firmas de minhash, vamos a usar (en este caso) veinte funciones de hash, las que intuitivamente jugarán el rol de una permutación de la matriz de elementos. 

In [4]:
### Contamos la cantidad de elementos totales que tenemos, 
### como el tamaño de la union de todos los conjuntos de shingles. 

un = set()
for (name,shingle) in odas_shingles.items():
        un = un.union(shingle)
        
print(len(un))

2349


La matriz de elementos tiene, entonces, 2349 filas. Las permutaciones serán funciones que transformen números entre 
0 y 2348 a numeros entre 0 y 2348. Definimos 50 de estas funciones. No son perfectas, pero van a ser suficientes para nuestro propósito. 

In [59]:
import random
h = []
n = 2349
p = 20063
num_hash = 50
for i in range(num_hash):
    a = random.randint(1,p-1)
    b = random.randint(1,p-1)
    h.append(crear_hash(a,b,p,n))


### Calculando la firma de minhash de las odas

Como ya sabemos, la matriz de elementos - conjuntos tiene 2349 filas, y 10 columnas, una por cada oda. Vamos a reducir esto a una matriz de 20 filas y 10 columnas: para cada columna, representando una oda, entregamos la firma de minhash correspondiente a la función de hash h[i].

In [60]:
### Creamos la matriz FH con las filas y columnas correspondientes. 
### Inicialmente la matriz tiene solo valores 3000, cualquier valor mas grande 
### que la cantidad de elementos sirve. 

n = len(odas_shingles)
FH = [[3000 for j in range(n)] for i in range(num_hash)]

### Tomamos la union de todos los elementos en el conjunto llamado un, 
### y para iterar sobre estos, lo transformamos a una lista ordenada

for (name,shingle) in odas_shingles.items():
    un = un.union(shingle)
    sorted_union = sorted(un)

### con esto podemos usar names[i] para conseguir el nombre de la i-esima oda. 
names = list(odas_shingles.keys())

             ### iteramos cada elemento
for e in range(0,len(sorted_union)): 
    ### iteramos cada columna, correspondiente a la cantidad de textos (odas en nuestro caso)  
    for i in range(0,n):
        ### si el elemento e no está en el texto i, no hacemos nada
        if sorted_union[e] not in odas_shingles[names[i]]:
            pass
        ### si el elemento si está, computamos los valores de h(e) y 
        else:
            for j in range(0,num_hash):
                FH[j][i] = min(FH[j][i],h[j](e))


In [53]:
### Ahora imprimimos la matriz
for row in FH:
    print(row)  

[2, 12, 9, 10, 2, 1, 3, 2, 0, 5]
[0, 3, 3, 7, 1, 0, 3, 3, 0, 3]
[1, 7, 4, 0, 7, 4, 1, 16, 1, 1]
[0, 1, 0, 0, 1, 6, 13, 2, 0, 2]
[0, 2, 0, 2, 1, 2, 0, 2, 2, 4]
[1, 1, 4, 1, 4, 1, 1, 1, 1, 4]
[0, 0, 6, 0, 0, 0, 0, 7, 4, 0]
[1, 5, 7, 5, 3, 5, 1, 5, 3, 7]
[5, 7, 3, 10, 0, 1, 4, 4, 5, 4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 2, 0, 0, 0, 4, 0, 0]
[0, 0, 9, 0, 0, 3, 0, 1, 0, 0]
[3, 3, 15, 11, 3, 7, 0, 11, 15, 3]
[1, 1, 1, 1, 1, 1, 1, 21, 3, 1]
[1, 1, 3, 1, 1, 1, 4, 1, 1, 7]
[6, 0, 14, 0, 0, 0, 6, 6, 0, 6]
[6, 10, 9, 2, 8, 1, 12, 10, 0, 11]
[1, 1, 1, 3, 1, 2, 6, 1, 1, 6]
[2, 1, 3, 1, 1, 1, 3, 5, 0, 0]
[4, 2, 2, 2, 2, 2, 4, 4, 0, 2]
[1, 5, 5, 1, 0, 1, 1, 5, 1, 1]
[0, 0, 1, 0, 0, 0, 1, 1, 0, 0]
[5, 5, 5, 5, 4, 4, 5, 5, 5, 5]
[1, 1, 1, 0, 1, 1, 1, 4, 1, 6]
[1, 0, 3, 1, 7, 4, 13, 12, 0, 4]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 1, 1, 1, 0, 1, 1, 1, 2, 1]
[1, 6, 3, 0, 6, 2, 4, 2, 2, 6]
[7, 7, 7, 0, 1, 1, 7, 7, 7, 7]
[1, 3, 6, 6, 1, 0, 0, 0, 1, 3]
[1, 3, 4, 1, 3, 1, 3, 

Ahora podemos pensar en que cada oda está representada por su firma de MinHash, que es 
la columna correspondiente, y podemos usar esa firma para revisar si acaso dos textos son similares. Así, si dos textos tienen la misma firma de minhash, entonces es probable que sean muy parecidos. 


Por ejemplo, estas son las columnas de las odas 5 y 9, correspondientes a los índices 4 y 8 de la matriz, que identificamos como similares antes: 

In [62]:
cuenta = 0
print(' 4     8 ')
print('---------')  
for row in FH:
    if row[4]==row[8]:
        cuenta += 1
    print('|'+str(row[4])+'|   |'+str(row[8])+'|')  
    
print('proporcion de la fila: '+str(cuenta/num_hash))


 4     8 
---------
|0|   |0|
|3|   |0|
|0|   |1|
|1|   |1|
|3|   |1|
|0|   |4|
|0|   |0|
|0|   |0|
|3|   |4|
|0|   |0|
|0|   |3|
|0|   |1|
|3|   |0|
|0|   |3|
|0|   |1|
|1|   |1|
|2|   |1|
|4|   |6|
|0|   |1|
|2|   |0|
|0|   |0|
|0|   |0|
|1|   |6|
|2|   |1|
|2|   |6|
|1|   |4|
|6|   |12|
|0|   |0|
|2|   |7|
|3|   |6|
|0|   |7|
|1|   |2|
|4|   |4|
|0|   |0|
|0|   |8|
|1|   |0|
|3|   |0|
|3|   |3|
|0|   |0|
|5|   |0|
|2|   |2|
|1|   |1|
|8|   |5|
|1|   |1|
|0|   |1|
|1|   |0|
|0|   |0|
|2|   |2|
|4|   |0|
|0|   |1|
proporcion de la fila: 0.36


Si bien hay algunas coincidencias, la firma no es igual. Esto es consistente con la similitud de Jaccard que vimos: si bien es la mas alta, no es un valor muy cercano a 1, si no que está cerca de 0.4. Esto coincide con la proporcion de hashes que son iguales en nuestro caso.  